In [10]:
import torch
import numpy as np
import os
#model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
#    in_channels=1, out_channels=1, init_features=32, pretrained=False)

In [11]:
#!pip install git+https://github.com/Rusteam/segmentation_models.pytorch.git@sam
import segmentation_models_pytorch as smp
model = smp.create_model("Unet", "sam-vit_b", encoder_weights="sa-1b", encoder_depth=4, decoder_channels=[256, 128, 64, 32])

/home/m256149/Documents/LeftVentricleSegmentation/code/sam/segmentation_models_pytorch/encoders/sam.py:121: UserWarning: Only 153 out of pretrained 177 SAM image encoder modules are loaded. Missing modules: []. Unused modules: ['blocks.0.attn.rel_pos_h', 'blocks.0.attn.rel_pos_w', 'blocks.1.attn.rel_pos_h', 'blocks.1.attn.rel_pos_w', 'blocks.2.attn.rel_pos_h', 'blocks.2.attn.rel_pos_w', 'blocks.3.attn.rel_pos_h', 'blocks.3.attn.rel_pos_w', 'blocks.4.attn.rel_pos_h', 'blocks.4.attn.rel_pos_w', 'blocks.5.attn.rel_pos_h', 'blocks.5.attn.rel_pos_w', 'blocks.6.attn.rel_pos_h', 'blocks.6.attn.rel_pos_w', 'blocks.7.attn.rel_pos_h', 'blocks.7.attn.rel_pos_w', 'blocks.8.attn.rel_pos_h', 'blocks.8.attn.rel_pos_w', 'blocks.9.attn.rel_pos_h', 'blocks.9.attn.rel_pos_w', 'blocks.10.attn.rel_pos_h', 'blocks.10.attn.rel_pos_w', 'blocks.11.attn.rel_pos_h', 'blocks.11.attn.rel_pos_w'].
  warnings.warn(


In [12]:
#model = list(model.children())
#w = model[0].weight
#model[0] = nn.Conv2d(1, 768, kernel_size=16, stride=16)
#model[0].weight = nn.Parameter(torch.mean(w, dim=1, keepdim=True))
#model = nn.Sequential(*model)

In [13]:
import matplotlib.pyplot as plt

In [14]:
import numpy as np
import torch
import torchio as tio
import os
import matplotlib.pyplot as plt
import json
import torch.nn as nn
from torchinfo import summary
import adabound
from sklearn import metrics
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix
from torch.optim import Adam,SGD

## Collect data

In [15]:
def get_trainbatch(traindir,labdir,batch_size):
    np.random.seed(0)
    #np.random.shuffle(traindir)
    return [(traindir[i:i + batch_size],labdir[i:i + batch_size]) for i in range(0, len(traindir), batch_size)]
device = 'cuda:0'
datadir = '/home/m256149/Documents/LeftVentricleSegmentation/data/Resources-4/'
np.random.seed(0)
for a,b,c in os.walk(datadir):
    break
tr = []
lb = []

In [16]:
for i in b:
    for q,fi,qw in os.walk(datadir+i+'/'):
        break
    for l in qw:
        if 'frame' in l:
            if 'gt' not in l:
                tr.append(datadir+i+'/'+l)
                lb.append(datadir+i+'/'+l[:-7]+'_gt'+'.nii.gz')

In [17]:
class diceloss(torch.nn.Module):
    def init(self):
        super(diceLoss, self).init()
    def forward(self,pred, target):
        smooth = 1.
        iflat = pred.contiguous().view(-1)
        tflat = target.contiguous().view(-1)
        intersection = (iflat * tflat).sum()
        A_sum = torch.sum(iflat * iflat)
        B_sum = torch.sum(tflat * tflat)
        return 1 - ((2. * intersection + smooth) / (A_sum + B_sum + smooth) )
class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()

    def forward(self, logits, targets):
        smooth = 1
        num = targets.size(0)
        """
               I am assuming the model does not have sigmoid layer in the end. if that is the case, change torch.sigmoid(logits) to simply logits
        """
        probs = logits#torch.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)

        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        
        
        # adding TV regularization
        tv_h = ((logits[:,:,1:,:] - logits[:,:,:-1,:]).pow(2)).sum()
        tv_w = ((logits[:,:,:,1:] - logits[:,:,:,:-1]).pow(2)).sum()    
       
        return score #+ 0.001*(tv_h + tv_w)

In [ ]:
train = tr
num_epochs = 1000
batch_size = 1
#warnings.filterwarnings("ignore")
#optimizer = adabound.AdaBound(vgg3d.parameters(), lr=0.4,weight_decay = 0.004,eps = 1e-4, final_lr=0.01)
#optimizer = SGD(model.parameters(), lr=.01,momentum=0.9)#,momentum=0.9,weight_decay=0.0005)#SGD(vgg3d.parameters(), lr=0.01,momentum=0.9,weight_decay=0.0005)
optimizer = Adam(model.parameters(), lr=.001)
criterion = SoftDiceLoss()#diceloss()#nn.BCEWithLogitsLoss() #nn.CrossEntropyLoss() ### momentum=0.9 diceloss()#

if torch.cuda.is_available():
    model = model.cuda(device = device)
    criterion = criterion.cuda(device = device)
val_losses = []
train_losses_ep = []
trans = tio.CropOrPad((3,1024,1024))
transout = tio.CropOrPad((1,1024,1024))

tRESCALE = tio.RescaleIntensity((-1,1))
for epoch in range(num_epochs):
    train_losses = []
    for batch,lab in get_trainbatch(train,lb,int(batch_size)):
        y_train = []
        for cnt,i in enumerate(batch):
            if cnt == 0:     
                orig = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
                y_train = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) ==2
                
                orig = tRESCALE(trans(orig))
                y_train = transout(y_train)
            else:
                temp = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
                temp_lab = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) == 2
                
                temp = tRESCALE(trans(temp))
                temp_lab = transout(temp_lab)
                
                #print(temp_lab.shape,temp.shape,orig.shape,y_train.shape)
                y_train = torch.cat((y_train,temp_lab),dim = 0)
                orig = torch.cat((orig,temp),dim = 0)
        
        
        orig = orig.float().cuda(device = device)
        y_train = y_train.float().cuda(device = device)
        
        temp_num = np.random.choice(orig.shape[0],2,replace = False)
        
        orig = orig[temp_num,:,:,:]
        y_train = y_train[temp_num,:,:,:]
        
        
        # clearing the Gradients of the model parameters
        optimizer.zero_grad()

        output_train = model(orig)
        loss_train = criterion(output_train, y_train) 
        # computing the updated weights of all the model parameters
        loss_train.backward()
        optimizer.step()
        train_losses.append(loss_train.item())
        del orig
        del y_train
    train_losses_ep.append(np.mean(train_losses))
    #if epoch%(num_epochs/25) == 0:
        # printing the validation loss
    print('Epoch : ',epoch+1, '\t', 'val loss :', '\t', 'train loss : ', np.mean(train_losses))

Epoch :  1 	 val loss : 	 train loss :  1.0041501492261886
Epoch :  2 	 val loss : 	 train loss :  1.0688541400432587
Epoch :  3 	 val loss : 	 train loss :  0.9998055854439736
Epoch :  4 	 val loss : 	 train loss :  0.9997897490859031
Epoch :  5 	 val loss : 	 train loss :  0.9981610748171806
Epoch :  6 	 val loss : 	 train loss :  0.9741992220282555
Epoch :  7 	 val loss : 	 train loss :  0.9724324229359627
Epoch :  8 	 val loss : 	 train loss :  0.9590441966056824
Epoch :  9 	 val loss : 	 train loss :  0.954014353454113
Epoch :  10 	 val loss : 	 train loss :  0.9486674031615258
Epoch :  11 	 val loss : 	 train loss :  0.9457015436887741
Epoch :  12 	 val loss : 	 train loss :  0.9446301352977753
Epoch :  13 	 val loss : 	 train loss :  0.9429363602399826
Epoch :  14 	 val loss : 	 train loss :  0.9308651474118232
Epoch :  15 	 val loss : 	 train loss :  0.9220324686169624
Epoch :  16 	 val loss : 	 train loss :  0.9348658537864685
Epoch :  17 	 val loss : 	 train loss :  0.9269217

Epoch :  138 	 val loss : 	 train loss :  0.9998675549030304
Epoch :  139 	 val loss : 	 train loss :  0.9998561808466911
Epoch :  140 	 val loss : 	 train loss :  0.999843764603138
Epoch :  141 	 val loss : 	 train loss :  0.9995532649755478
Epoch :  142 	 val loss : 	 train loss :  0.9991085371375084
Epoch :  143 	 val loss : 	 train loss :  0.9989633059501648
Epoch :  144 	 val loss : 	 train loss :  0.9988516092300415
Epoch :  145 	 val loss : 	 train loss :  0.9987598884105683
Epoch :  146 	 val loss : 	 train loss :  0.9986681360006332
Epoch :  147 	 val loss : 	 train loss :  0.998558396100998
Epoch :  148 	 val loss : 	 train loss :  0.9984358912706375
Epoch :  149 	 val loss : 	 train loss :  0.9909715619683266
Epoch :  150 	 val loss : 	 train loss :  1.0002916908264161
Epoch :  151 	 val loss : 	 train loss :  0.999891360104084
Epoch :  152 	 val loss : 	 train loss :  0.9985039430856705
Epoch :  153 	 val loss : 	 train loss :  1.0193736293911935
Epoch :  154 	 val loss : 	

Epoch :  273 	 val loss : 	 train loss :  0.9999333497881889
Epoch :  274 	 val loss : 	 train loss :  0.9999108901619911
Epoch :  275 	 val loss : 	 train loss :  0.9998690712451935
Epoch :  276 	 val loss : 	 train loss :  0.9999930796027183
Epoch :  277 	 val loss : 	 train loss :  0.9996160167455673
Epoch :  278 	 val loss : 	 train loss :  0.9999848246574402
Epoch :  279 	 val loss : 	 train loss :  0.9999615204334259
Epoch :  280 	 val loss : 	 train loss :  0.9999544975161553
Epoch :  281 	 val loss : 	 train loss :  0.9999472188949585
Epoch :  282 	 val loss : 	 train loss :  0.9999354249238968
Epoch :  283 	 val loss : 	 train loss :  0.9998079219460487
Epoch :  284 	 val loss : 	 train loss :  1.000060768723488
Epoch :  285 	 val loss : 	 train loss :  0.9999079605937005
Epoch :  286 	 val loss : 	 train loss :  0.999546868801117
Epoch :  287 	 val loss : 	 train loss :  1.0000001123547555
Epoch :  288 	 val loss : 	 train loss :  0.9998498484492302
Epoch :  289 	 val loss : 

Epoch :  409 	 val loss : 	 train loss :  1.0012299436330796
Epoch :  410 	 val loss : 	 train loss :  1.001050528883934
Epoch :  411 	 val loss : 	 train loss :  1.000899893641472
Epoch :  412 	 val loss : 	 train loss :  1.0007381016016006
Epoch :  413 	 val loss : 	 train loss :  1.0005737006664277
Epoch :  414 	 val loss : 	 train loss :  1.000434119105339
Epoch :  415 	 val loss : 	 train loss :  1.0002961128950119
Epoch :  416 	 val loss : 	 train loss :  1.000174520611763
Epoch :  417 	 val loss : 	 train loss :  1.0001095843315124
Epoch :  418 	 val loss : 	 train loss :  1.0000771331787108
Epoch :  419 	 val loss : 	 train loss :  1.0000538820028304
Epoch :  420 	 val loss : 	 train loss :  1.0000305271148682
Epoch :  421 	 val loss : 	 train loss :  1.0000005039572715
Epoch :  422 	 val loss : 	 train loss :  0.9999632692337036
Epoch :  423 	 val loss : 	 train loss :  0.9999042493104935
Epoch :  424 	 val loss : 	 train loss :  0.9997094181180001
Epoch :  425 	 val loss : 	 

Epoch :  544 	 val loss : 	 train loss :  1.0001502895355225
Epoch :  545 	 val loss : 	 train loss :  1.000132001042366
Epoch :  546 	 val loss : 	 train loss :  1.0001162278652191
Epoch :  547 	 val loss : 	 train loss :  1.0001021260023117
Epoch :  548 	 val loss : 	 train loss :  1.0000895392894744
Epoch :  549 	 val loss : 	 train loss :  1.00007807970047
Epoch :  550 	 val loss : 	 train loss :  1.0000678688287734
Epoch :  551 	 val loss : 	 train loss :  1.0000587892532349
Epoch :  552 	 val loss : 	 train loss :  1.0000504672527313
Epoch :  553 	 val loss : 	 train loss :  1.0000422435998917
Epoch :  554 	 val loss : 	 train loss :  1.000034448504448
Epoch :  555 	 val loss : 	 train loss :  1.0000267183780671
Epoch :  556 	 val loss : 	 train loss :  1.0000197333097458
Epoch :  557 	 val loss : 	 train loss :  1.0000137984752655
Epoch :  558 	 val loss : 	 train loss :  1.000008028447628
Epoch :  559 	 val loss : 	 train loss :  1.0000022372603417


In [ ]:
torch.save(model.state_dict(),'/home/m256149/Documents/LeftVentricleSegmentation/code/unet-sam.pt')

In [ ]:
for i in range(100):
    plt.figure(i)
    
    plt.imshow(model(orig).cpu().detach().numpy()[i,0,:,:])
    plt.figure(i+100)
    plt.imshow(y_train.cpu()[i,0,:,:])

In [ ]:
sum(sum(model(orig) ==1)).sum() , sum(sum(model(orig) ==0)).sum()

In [ ]:
model(orig[0,:,:,:][None])

In [ ]:
for batch,lab in get_trainbatch(train,lb,int(batch_size)):
    y_train = []
    for cnt,i in enumerate(batch):
        if cnt == 0:     
            orig = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
            y_train = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) ==2

            orig = tRESCALE(trans(orig))
            y_train = trans(y_train)
        else:
            temp = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
            temp_lab = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) == 2

            temp = tRESCALE(trans(temp))
            temp_lab = trans(temp_lab)

            #print(temp_lab.shape,temp.shape,orig.shape,y_train.shape)
            y_train = torch.cat((y_train,temp_lab),dim = 0)
            orig = torch.cat((orig,temp),dim = 0)

In [ ]:
orig = orig.float().cuda(device = device)
y_train = y_train.float().cuda(device = device)


In [ ]:
for i in range(100):
    plt.figure(i)
    
    plt.imshow(model(orig).cpu().detach().numpy()[i,0,:,:])
    plt.figure(i+100)
    plt.imshow(y_train.cpu()[i,0,:,:])
    
    plt.figure(i+200)
    plt.imshow(orig.cpu().detach().numpy()[i,0,:,:],vmin = -1,vmax = 0)

## Read test data

In [ ]:
datadir = '/home/m256149/Documents/LeftVentricleSegmentation/Resources-5/'
#fat,water,study
for a,b,c in os.walk(datadir):
    break
tr = []
lb = []
for i in b:
    for q,fi,qw in os.walk(datadir+i+'/'):
        break
    for l in qw:
        if 'frame' in l:
            if 'gt' not in l:
                tr.append(datadir+i+'/'+l)
                lb.append(datadir+i+'/'+l[:-7]+'_gt'+'.nii.gz')

## calculate dice score on test set

In [ ]:
model.eval()
ds = 0
num_slice = 0 
batch_size = 1
for batch,lab in get_trainbatch(tr,lb,int(batch_size)):
    y_test = []
    for cnt,i in enumerate(batch):
        if cnt == 0:     
            orig = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
            y_test = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) ==2

            orig = tRESCALE(trans(orig))
            y_test = trans(y_test)
        else:
            temp = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
            temp_lab = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) == 2

            temp = tRESCALE(trans(temp))
            temp_lab = trans(temp_lab)

            #print(temp_lab.shape,temp.shape,orig.shape,y_train.shape)
            y_test = torch.cat((y_test,temp_lab),dim = 0)
            orig = torch.cat((orig,temp),dim = 0)


    orig = orig.float().cuda(device = device)
    y_test = y_test.float().cuda(device = device)
    ds += criterion(torch.round(model(orig)),y_test)*y_test.shape[0]
    num_slice += y_test.shape[0]
    del orig
    del y_test
print('average dice score', ds/num_slice)

In [ ]:
torch.save(model.state_dict(),'/home/m256149/Documents/LeftVentricleSegmentation/unet-sam.pt')

In [ ]:
trainedmodel = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=1, out_channels=1, init_features=32, pretrained=False)
trainedmodel.load_state_dict(torch.load('/home/m256149/Documents/LeftVentricleSegmentation/unet.pt'))

In [ ]:
trainedmodel.eval()
ds = 0
num_slice = 0 
batch_size = 8
for batch,lab in get_trainbatch(tr,lb,int(batch_size)):
    y_test = []
    for cnt,i in enumerate(batch):
        if cnt == 0:     
            orig = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
            y_test = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) ==2

            orig = tRESCALE(trans(orig))
            y_test = trans(y_test)
        else:
            temp = tio.ScalarImage(batch[cnt]).data.permute(3,0,1,2)
            temp_lab = tio.ScalarImage(lab[cnt]).data.permute(3,0,1,2) == 2

            temp = tRESCALE(trans(temp))
            temp_lab = trans(temp_lab)

            #print(temp_lab.shape,temp.shape,orig.shape,y_train.shape)
            y_test = torch.cat((y_test,temp_lab),dim = 0)
            orig = torch.cat((orig,temp),dim = 0)


    orig = orig.float()#.cuda(device = device)
    y_test = y_test.float()#.cuda(device = device)
    ds += criterion(torch.round(trainedmodel(orig)),y_test).item()*y_test.shape[0]
    num_slice += y_test.shape[0]
    print(y_test.shape[0])
    del orig
    del y_test
print('average dice score', ds/num_slice)

## save masks as nifti

In [ ]:
trainedmodel.eval()
i = 0
for batch,lab in zip(tr,lb):
    orig = tio.ScalarImage(batch).data.permute(3,0,1,2)
    orig = tRESCALE(trans(orig))

    orig = orig.float()#.cuda(device = device)

    maskdata = torch.round(trainedmodel(orig)).permute(1,2,3,0).detach()
    
   
    y_test = tio.ScalarImage(lab).data.permute(3,0,1,2) ==2
    y_test = trans(y_test)
    
    
    # save orig, maskdata, y_test
    temp = tio.ScalarImage(batch)
    temp.data = orig.permute(1,2,3,0).detach()
    temp.save(batch[:-14] + batch[-9:-7] +'resize.nii.gz')
    
    
    
    temp = tio.ScalarImage(lab)
    temp.data = maskdata
    temp.save(lab[:-10] + '_outgt.nii.gz')
    
    temp = tio.ScalarImage(lab)
    temp.data = y_test.permute(1,2,3,0).detach()
    temp.save(lab[:-10] + '_resizegt.nii.gz')


    
    del orig
    del y_test
    i += 1